### **Step 1- Uploading dataset.zip**

In [1]:
from google.colab import files
uploaded = files.upload()

Saving dataset.zip to dataset.zip


### **Step 2- Installing MFA via conda**

In [2]:
import os
os.chdir('/content')

!wget -q https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!bash Miniconda3-latest-Linux-x86_64.sh -b -p /usr/local/miniconda3

!/usr/local/miniconda3/bin/conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/main
!/usr/local/miniconda3/bin/conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/r

os.environ['PATH'] = '/usr/local/miniconda3/bin:' + os.environ['PATH']

!/usr/local/miniconda3/bin/conda install -y -c conda-forge montreal-forced-aligner

PREFIX=/usr/local/miniconda3
Unpacking bootstrapper...
Unpacking payload...

Installing base environment...

Preparing transaction: ...working... done
Executing transaction: ...working... done
installation finished.
    You currently have a PYTHONPATH environment variable set. This may cause
    unexpected behavior when running the Python interpreter in Miniconda3.
    For best results, please verify that your PYTHONPATH only points to
    directories of packages that are compatible with the Python interpreter
    in Miniconda3: /usr/local/miniconda3
accepted Terms of Service for https://repo.anaconda.com/pkgs/main
accepted Terms of Service for https://repo.anaconda.com/pkgs/r
Jupyter detected...
2 channel Terms of Service accepted
Retrieving notices: - \ | / done
Channels:
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: | / - \ | / - \ | / - \ | done

## Package Plan ##

  environment location: /usr/local/miniconda3

  added / updat

### **Step 3- Preparing corpus **

In [3]:
import zipfile, shutil, os
from pathlib import Path
import re


def clean_text_for_mfa(text):
    text = re.sub(r'\s+', ' ', text)
    text = text.upper()
    text = re.sub(r'[^A-Z ]', '', text)
    return text.strip()


zip_path = next(iter(uploaded))
if Path('/content/dataset').exists():
    shutil.rmtree('/content/dataset')
with zipfile.ZipFile(zip_path) as z:
    z.extractall('/content/dataset')


if Path('/content/corpus').exists():
    shutil.rmtree('/content/corpus')
cdir = Path('/content/corpus')
cdir.mkdir(parents=True, exist_ok=True)

print(f"Files in {cdir} BEFORE population: {list(cdir.iterdir())}")


def normalize_stem(s):
    return s.replace(" - Copy", "").replace(" - copy", "").casefold()


w_src_dir = None
t_src_dir = None
for root, dirs, files in os.walk('/content/dataset'):
    if 'wav' in dirs:
        w_src_dir = Path(root) / 'wav'
    if 'transcripts' in dirs:
        t_src_dir = Path(root) / 'transcripts'


if not w_src_dir:
    w_src_dir = Path('/content/dataset/Assignment/wav')
if not t_src_dir:
    t_src_dir = Path('/content/dataset/Assignment/transcripts')


if not w_src_dir.exists():
    wav_files = list(Path('/content/dataset').rglob('*.wav'))
    if wav_files:
        w_src_dir = wav_files[0].parent
if not t_src_dir.exists():
    txt_files = list(Path('/content/dataset').rglob('*.txt')) + list(Path('/content/dataset').rglob('*.TXT'))
    if txt_files:
        t_src_dir = txt_files[0].parent

print(f"Source WAV dir: {w_src_dir}")
print(f"Source Transcripts dir: {t_src_dir}")


transcripts_map = {}
if t_src_dir and t_src_dir.exists():
    for p in list(t_src_dir.glob("*.txt")) + list(t_src_dir.glob("*.TXT")):
        normalized_stem = normalize_stem(p.stem)
        transcripts_map[normalized_stem] = p
else:
    print("WARNING: Transcript source directory (t_src_dir) does not exist or was not found.")


paired, missing = 0, 0
if w_src_dir and w_src_dir.exists():
    processed_stems = set()
    for w in sorted(w_src_dir.glob("*.wav")):
        original_stem = w.stem
        normalized_stem = normalize_stem(original_stem)

        if normalized_stem in processed_stems:
            print(f"Skipping duplicate processed stem: {normalized_stem} (from {original_stem}.wav)")
            continue

        t = transcripts_map.get(normalized_stem)

        if not t:
            print(f"Missing transcript for WAV: {original_stem}.wav (normalized to {normalized_stem})")
            missing += 1
            continue


        shutil.copy2(w, cdir / f"{normalized_stem}.wav")


        text = t.read_text(encoding="utf-8", errors="ignore")
        cleaned_text = clean_text_for_mfa(text)
        (cdir / f"{normalized_stem}.lab").write_text(cleaned_text + "\n", encoding="utf-8")


        paired += 1
        processed_stems.add(normalized_stem)
else:
    print("WARNING: WAV source directory (w_src_dir) does not exist or was not found.")

print("corpus target:", cdir)
print("paired:", paired, "missing:", missing)
print(f"Files in {cdir} AFTER population: {list(cdir.iterdir())}")

Files in /content/corpus BEFORE population: []
Source WAV dir: /content/dataset/wav
Source Transcripts dir: /content/dataset/transcripts
Skipping duplicate processed stem: isle_sess0131_blockd02_01_sprt1 (from ISLE_SESS0131_BLOCKD02_01_sprt1.wav)
Skipping duplicate processed stem: isle_sess0131_blockd02_02_sprt1 (from ISLE_SESS0131_BLOCKD02_02_sprt1.wav)
Skipping duplicate processed stem: isle_sess0131_blockd02_03_sprt1 (from ISLE_SESS0131_BLOCKD02_03_sprt1.wav)
corpus target: /content/corpus
paired: 6 missing: 0
Files in /content/corpus AFTER population: [PosixPath('/content/corpus/f2bjrlp1.wav'), PosixPath('/content/corpus/f2bjrlp2.wav'), PosixPath('/content/corpus/isle_sess0131_blockd02_01_sprt1.lab'), PosixPath('/content/corpus/f2bjrlp1.lab'), PosixPath('/content/corpus/isle_sess0131_blockd02_03_sprt1.lab'), PosixPath('/content/corpus/isle_sess0131_blockd02_03_sprt1.wav'), PosixPath('/content/corpus/isle_sess0131_blockd02_01_sprt1.wav'), PosixPath('/content/corpus/isle_sess0131_blo

### **Step 4- Check structure**

In [4]:
import os
from pathlib import Path

cdir = Path('/content/corpus')

print("Files in corpus (flat structure):")
for f in sorted(cdir.glob("*")):
    print(f.name, f.stat().st_size)

wavs = set(f.stem for f in cdir.glob("*.wav"))
labs = set(f.stem for f in cdir.glob("*.lab"))
print(f"\nWAV files: {len(wavs)}")
print(f"LAB files: {len(labs)}")
print(f"Matched pairs: {len(wavs & labs)}")

Files in corpus (flat structure):
f2bjrlp1.lab 412
f2bjrlp1.wav 810916
f2bjrlp2.lab 478
f2bjrlp2.wav 917742
f2bjrlp3.lab 526
f2bjrlp3.wav 983642
isle_sess0131_blockd02_01_sprt1.lab 22
isle_sess0131_blockd02_01_sprt1.wav 136096
isle_sess0131_blockd02_02_sprt1.lab 18
isle_sess0131_blockd02_02_sprt1.wav 128096
isle_sess0131_blockd02_03_sprt1.lab 19
isle_sess0131_blockd02_03_sprt1.wav 148096

WAV files: 6
LAB files: 6
Matched pairs: 6


### **Step 5: Downloading models**

In [5]:

!/usr/local/miniconda3/bin/mfa model download dictionary english_us_arpa
!/usr/local/miniconda3/bin/mfa model download acoustic english_us_arpa

### **Step 6: Running alignment**

In [6]:
!ls -l /content/corpus
!/usr/local/miniconda3/bin/mfa align /content/corpus english_us_arpa english_us_arpa /content/aligned -v

total 3092
-rw-r--r-- 1 root root    412 Nov  7 09:19 f2bjrlp1.lab
-rw-r--r-- 1 root root 810916 Nov  7 09:19 f2bjrlp1.wav
-rw-r--r-- 1 root root    478 Nov  7 09:19 f2bjrlp2.lab
-rw-r--r-- 1 root root 917742 Nov  7 09:19 f2bjrlp2.wav
-rw-r--r-- 1 root root    526 Nov  7 09:19 f2bjrlp3.lab
-rw-r--r-- 1 root root 983642 Nov  7 09:19 f2bjrlp3.wav
-rw-r--r-- 1 root root     22 Nov  7 09:19 isle_sess0131_blockd02_01_sprt1.lab
-rw-r--r-- 1 root root 136096 Nov  7 09:19 isle_sess0131_blockd02_01_sprt1.wav
-rw-r--r-- 1 root root     18 Nov  7 09:19 isle_sess0131_blockd02_02_sprt1.lab
-rw-r--r-- 1 root root 128096 Nov  7 09:19 isle_sess0131_blockd02_02_sprt1.wav
-rw-r--r-- 1 root root     19 Nov  7 09:19 isle_sess0131_blockd02_03_sprt1.lab
-rw-r--r-- 1 root root 148096 Nov  7 09:19 isle_sess0131_blockd02_03_sprt1.wav
 INFO     Setting up corpus information...                                      
 INFO     Loading corpus from source files...                                   
   6% ━━╺━━━━━━━━

### **Step 7: Creating zip folder and downloading it**

In [7]:
!cd /content && zip -r aligned_textgrids.zip aligned
from google.colab import files
files.download('/content/aligned_textgrids.zip')

  adding: aligned/ (stored 0%)
  adding: aligned/f2bjrlp3.TextGrid (deflated 89%)
  adding: aligned/f2bjrlp1.TextGrid (deflated 89%)
  adding: aligned/isle_sess0131_blockd02_03_sprt1.TextGrid (deflated 83%)
  adding: aligned/f2bjrlp2.TextGrid (deflated 89%)
  adding: aligned/isle_sess0131_blockd02_02_sprt1.TextGrid (deflated 84%)
  adding: aligned/alignment_analysis.csv (deflated 48%)
  adding: aligned/isle_sess0131_blockd02_01_sprt1.TextGrid (deflated 84%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>